## Imports

In [ ]:
import os
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Convolution2D, Activation, MaxPooling2D, Dense
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50

## Datagen object creation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
##image aug datagen
datagen_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255., # Rescaling
    rotation_range = 40, # for augmentation
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_dir = 'C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset/train'
test_dir = 'C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset/test'

## Loading image without optimized data pipeline


In [ ]:
image_size = [256, 256]

In [ ]:
##augmented datagen
training_set = datagen_aug.flow_from_directory('C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset/train',
                                          batch_size=64,
                                           target_size = image_size,
                                            shuffle=True,
                                          class_mode='categorical')

In [ ]:
##plot images
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
    
    
augmented_images = [training_set[0][0][0] for i in range(10)]
plotImages(augmented_images)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset/test',
                                          batch_size=32,
                                        target_size = image_size,
                                          class_mode='categorical')

In [ ]:
augmented_images = [test_set[0][0][0] for i in range(10)]
plotImages(augmented_images)

In [ ]:
labels = (training_set.class_indices)
labels

In [ ]:
len(labels)

## Build Model

In [ ]:
resnet = tf.keras.applications.ResNet50V2(
    input_shape=image_size + [3],
    include_top=False,
    weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
)

In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(labels), activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
start = time.time()
history = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
end = time.time()
print("Time taken:", end - start)

In [ ]:

# plot the loss
# plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
# plt.legend()
# plt.show()
# plt.savefig('LossVal_loss')

# # plot the accuracy
# plt.plot(r.history['accuracy'], label='train acc')
# plt.plot(r.history['val_accuracy'], label='val acc')
# plt.legend()
# plt.show()
# plt.savefig('AccVal_acc')